In [5]:
import pandas as pd
import numpy as np
import pickle
import branca.colormap
from collections import defaultdict
import folium
import folium.plugins as plugins

In [6]:
df = pd.read_csv('./France_Hospital_data/donnees-hospitalieres-nouveaux-covid19-2020-04-23-19h00.csv', header=0,sep=';')
df = df.groupby(['jour',  'dep']).sum()
time_index = list(np.unique(df.index.get_level_values(0)))

In [7]:
with open('./processed_data/departments.pkl', 'rb') as handle:
    France_atts = pickle.load(handle)
loci = pd.DataFrame.from_dict(France_atts)
loci.columns = np.unique(df.index.get_level_values(1))

In [9]:
for cat in df.columns:
    data_hosp = []
    for _, df_temp in df.groupby(level=0):
        data_temp = []
        for weight, (key, value) in zip(df_temp[cat], loci.loc['capital_coords'].items()):
            data_temp.append([value['lat'], value['lng'], weight/(df[cat].max())])
        data_hosp.append(data_temp)
    
    m = folium.Map([48., 5.], tiles='stamentoner', zoom_start=6)

    hm = plugins.HeatMapWithTime(
        data_hosp,
        radius=45,
        index=time_index,
        auto_play=True,
        max_opacity=0.3
    )
    hm.add_to(m)
    
    file_path = r"./visualisations/{}.html".format(str(cat))
    hm.save(file_path)